### 1. Create a new database with a table named Roster that has three fields: Name, Species, and Age. The Name and Species columns should be text fields, and the Age column should be an integer field.

In [ ]:
import pyodbc

def ensure_roster_db(
    server: str = r"localhost\SQLEXPRESS",   # change for your host\instance
    database: str = "PetClub",
    use_trusted_connection: bool = True,
    username: str | None = None,
    password: str | None = None,
):
    """
    Connect to SQL Server, create <database> and dbo.Roster if they don't exist.

    Parameters
    ----------
    server : str
        Host\instance or IP; e.g. 'localhost\\SQLEXPRESS' or '192.168.1.10'.
    database : str
        Database name to create/use.
    use_trusted_connection : bool
        If True, use Windows auth; otherwise supply username/password.
    username, password : str | None
        SQL log‑in credentials (ignored when use_trusted_connection is True).
    """
    # Build the connection string
    parts = [
        "DRIVER={ODBC Driver 17 for SQL Server}",
        f"SERVER={server}",
        "DATABASE=master",                 # start in master; we’ll switch later
    ]
    if use_trusted_connection:
        parts.append("Trusted_Connection=yes")
    else:
        parts.append(f"UID={username}")
        parts.append(f"PWD={password}")
    conn_str = ";".join(parts) + ";"

    with pyodbc.connect(conn_str, autocommit=True) as conn:
        cur = conn.cursor()

        # 1️⃣  Create the database if it doesn't exist
        cur.execute(f"IF DB_ID(N'{database}') IS NULL CREATE DATABASE [{database}];")
        print(f"✅ Database '{database}' ready (or already existed).")

        # 2️⃣  Create the table if it doesn't exist
        cur.execute(f"USE [{database}];")
        cur.execute("""
            IF OBJECT_ID(N'dbo.Roster', N'U') IS NULL
            CREATE TABLE dbo.Roster
            (
                Name     NVARCHAR(100) NOT NULL,
                Species  NVARCHAR(50)  NOT NULL,
                Age      INT           NOT NULL
            );
        """)
        print("✅ Table dbo.Roster is ready!")

# ------------------------------------------------------------------
# HOW TO USE
# ------------------------------------------------------------------
if __name__ == "__main__":
    ensure_roster_db(
        server=r"localhost\SQLEXPRESS",  # replace with your server
        use_trusted_connection=True      # or False + username/password
    )


### 2. Populate your new table with the following values:

Name	Species	Age
Benjamin Sisko	Human	40
Jadzia Dax	Trill	300
Kira Nerys	Bajoran	29

In [ ]:
import pyodbc

# --- build a connection string that points directly at the PetClub DB ---
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\\SQLEXPRESS;"     # adjust if different
    "DATABASE=PetClub;"
    "Trusted_Connection=yes;"           # or UID=...;PWD=...
)

rows = [
    ("Benjamin Sisko", "Human",   40),
    ("Jadzia Dax",     "Trill",  300),
    ("Kira Nerys",     "Bajoran", 29),
]

with pyodbc.connect(conn_str) as conn:
    cur = conn.cursor()
    cur.executemany(
        "INSERT INTO dbo.Roster (Name, Species, Age) VALUES (?, ?, ?);",
        rows
    )
    conn.commit()
print("Roster populated!")


### 3. Update the Name of Jadzia Dax to be Ezri Dax

In [ ]:
import pyodbc

conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\\SQLEXPRESS;"  # adjust to your server
    "DATABASE=PetClub;"
    "Trusted_Connection=yes;"        # or use UID=...;PWD=...
)

with pyodbc.connect(conn_str) as conn:
    cur = conn.cursor()
    cur.execute("""
        UPDATE dbo.Roster
        SET Name = ?
        WHERE Name = ?;
    """, ("Ezri Dax", "Jadzia Dax"))
    conn.commit()

print("Name updated successfully.")


### 4. Display the Name and Age of everyone in the table classified as Bajoran.

In [ ]:
import pyodbc

conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\\SQLEXPRESS;"    # update if needed
    "DATABASE=PetClub;"
    "Trusted_Connection=yes;"
)

with pyodbc.connect(conn_str) as conn:
    cur = conn.cursor()
    cur.execute("""
        SELECT Name, Age
        FROM dbo.Roster
        WHERE Species = ?;
    """, ("Bajoran",))
    
    for row in cur.fetchall():
        print(f"Name: {row.Name}, Age: {row.Age}")
